<a href="https://colab.research.google.com/github/peterbabulik/ETA/blob/main/QuantumPhaseEstimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### The Eigenvalue Equation

**The Math:** Finding the phase (eigenvalue) of a matrix operator.
$$ U|\psi\rangle = e^{i\theta} |\psi\rangle $$

Where $U$ is a unitary matrix, $|\psi\rangle$ is an eigenvector, and $\theta$ is the phase we want to find.

### The Quantum Translation: Quantum Phase Estimation (QPE)

This is the sub-routine inside **Shor's Algorithm** (factoring) and Quantum Chemistry simulations.

**The Logic:**
1. We have a "Target" qubit in state $|\psi\rangle$ and "Counting" qubits initialized to $|0\rangle$.
2. We apply Hadamard gates to counting qubits (create superposition).
3. We apply "Controlled-U" operations with increasing powers ($U^1, U^2, U^4, U^8...$).
4. The phase $\theta$ gets "kicked back" onto the counting qubits.
5. We run **Inverse QFT** on the counting qubits to read $\theta$ as a binary number.

**Analogy for Python Devs:**
Think of this as `detect_frequency()`. You have a black-box function $U$. You ping it multiple times and use interference to determine its internal rotation frequency.

### The Qiskit Implementation

We'll estimate the phase of a simple rotation gate $R_z(\theta)$. The eigenstate of $R_z$ is $|1\rangle$, and the eigenvalue is $e^{i\theta/2}$.

In [1]:
!pip install qiskit qiskit-aer -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 2.2 MB/s eta 0:00:00


In [2]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

def build_qpe_circuit(counting_qubits, target_angle):
    """
    Builds a Quantum Phase Estimation circuit.

    Args:
        counting_qubits: Number of qubits for precision (n bits = precision of 2^-n)
        target_angle: The rotation angle of our unitary U = Rz(angle)

    Returns:
        QuantumCircuit: The QPE circuit
    """
    # Total qubits = counting qubits + 1 target qubit
    n_count = counting_qubits
    n_target = 1
    total_qubits = n_count + n_target

    # Create circuit with classical bits for measurement
    qc = QuantumCircuit(total_qubits, n_count)

    # -------------------------------------------------------
    # STEP 1: Initialize Target Qubit to Eigenstate |1>
    # For Rz gate, |1> is an eigenstate
    # -------------------------------------------------------
    target_qubit = total_qubits - 1
    qc.x(target_qubit)  # Flip to |1>

    # -------------------------------------------------------
    # STEP 2: Apply Hadamard to Counting Qubits
    # Creates superposition: (|0> + |1>)/sqrt(2) for each
    # -------------------------------------------------------
    for i in range(n_count):
        qc.h(i)

    qc.barrier()

    # -------------------------------------------------------
    # STEP 3: Apply Controlled-U Operations (Phase Kickback)
    # Each counting qubit controls U^(2^j)
    # The phase gets "kicked back" to the control qubit
    # -------------------------------------------------------
    for j in range(n_count):
        # Power of U: U^(2^j)
        # For Rz(angle), applying it 2^j times gives Rz(angle * 2^j)
        power = 2 ** j
        angle_power = target_angle * power

        # Controlled-Rz (decomposed into CNOTs and Rz)
        # C-Rz(θ) = CNOT - Rz(θ) - CNOT (on target, controlled by counting qubit)
        # Actually, we use cp (controlled-phase) which is equivalent
        qc.cp(angle_power, j, target_qubit)

    qc.barrier()

    # -------------------------------------------------------
    # STEP 4: Apply Inverse QFT on Counting Qubits
    # This converts the phase information into a binary number
    # -------------------------------------------------------
    # Build QFT and invert it
    def build_qft(n_qubits):
        qft = QuantumCircuit(n_qubits)
        for i in range(n_qubits):
            qft.h(i)
            for j in range(i + 1, n_qubits):
                power = j - i + 1
                angle = np.pi / (2 ** power)
                qft.cp(angle, j, i)
        # Swap qubits
        for i in range(n_qubits // 2):
            qft.swap(i, n_qubits - i - 1)
        return qft

    qft = build_qft(n_count)
    inv_qft = qft.inverse()

    # Add inverse QFT to circuit
    qc.compose(inv_qft, qubits=range(n_count), inplace=True)

    qc.barrier()

    # -------------------------------------------------------
    # STEP 5: Measure Counting Qubits
    # The result is the phase θ encoded in binary
    # -------------------------------------------------------
    qc.measure(range(n_count), range(n_count))

    return qc

# --- Example Usage ---

# We want to find phase θ = 1/4 (in units of 2π)
# So our unitary U = Rz(2π * θ) = Rz(π/2)
# Expected measurement: 01 (binary for 1/4 = 0.25)

target_phase = 1/4  # θ = 0.25 (we want to find this)
target_angle = 2 * np.pi * target_phase  # Convert to radians
n_counting_qubits = 3  # 3 bits of precision

# Build and run circuit
qc = build_qpe_circuit(n_counting_qubits, target_angle)

simulator = AerSimulator()
compiled_circuit = transpile(qc, simulator)
job = simulator.run(compiled_circuit, shots=1000)
result = job.result()
counts = result.get_counts()

# --- Analyze Results ---
print("--- Quantum Phase Estimation Results ---")
print(f"Target Phase (θ): {target_phase} (should be 0.25)")
print(f"Counting Qubits: {n_counting_qubits}")
print(f"Precision: 1/{2**n_counting_qubits} = {1/2**n_counting_qubits:.4f}")
print(f"\nMeasurement Counts: {counts}")

# Find most probable result
most_probable = max(counts, key=counts.get)
measured_decimal = int(most_probable, 2)
measured_phase = measured_decimal / (2 ** n_counting_qubits)

print(f"\nMost Probable Outcome: |{most_probable}⟩")
print(f"Measured Phase: {measured_phase:.4f}")
print(f"Expected Phase: {target_phase:.4f}")
print(f"Error: {abs(measured_phase - target_phase):.4f}")

print("\n--- Circuit Diagram ---")
print(qc.draw(output='text', fold=80))

--- Quantum Phase Estimation Results ---
Target Phase (θ): 0.25 (should be 0.25)
Counting Qubits: 3
Precision: 1/8 = 0.1250

Measurement Counts: {'101': 2, '100': 73, '110': 279, '010': 431, '011': 74, '111': 141}

Most Probable Outcome: |010⟩
Measured Phase: 0.2500
Expected Phase: 0.2500
Error: 0.0000

--- Circuit Diagram ---
     ┌───┐ ░                          ░                                  »
q_0: ┤ H ├─░──■───────────────────────░──X──────────────────────■────────»
     ├───┤ ░  │                       ░  │                ┌───┐ │        »
q_1: ┤ H ├─░──┼────────■──────────────░──┼───────■────────┤ H ├─┼────────»
     ├───┤ ░  │        │              ░  │ ┌───┐ │P(-π/4) └───┘ │P(-π/8) »
q_2: ┤ H ├─░──┼────────┼──────■───────░──X─┤ H ├─■──────────────■────────»
     ├───┤ ░  │P(π/2)  │P(π)  │P(2π)  ░    └───┘                         »
q_3: ┤ X ├─░──■────────■──────■───────░──────────────────────────────────»
     └───┘ ░                          ░                                

### Understanding the Translation

1. **$U|\psi\rangle = e^{i\theta}|\psi\rangle$ (The Eigenvalue Equation):**
   - In the code, $U$ is our rotation gate `Rz(angle)`.
   - $|\psi\rangle$ is the eigenstate `|1⟩`.
   - $\theta$ is what we're trying to find.

2. **Phase Kickback:**
   - When we apply controlled-$U$, the phase $e^{i\theta}$ gets "kicked back" to the control qubit.
   - This is the quantum magic that makes QPE work.

3. **Inverse QFT:**
   - The phase information is stored in the relative phases of the counting qubits.
   - Inverse QFT converts these phases into a readable binary number.

### Why is this useful?

**Shor's Algorithm:** QPE is the core subroutine that finds the period of a function, which leads to factoring numbers efficiently.

**Quantum Chemistry:** QPE can find energy eigenvalues of molecular Hamiltonians, enabling simulation of chemical reactions.

**Precision:** With $n$ counting qubits, you get $n$ bits of precision. Want more precision? Add more qubits!